In [1]:
import random
from pathlib import Path
from typing import Final

import pandas as pd

from magn.database.database import Database
from magn.database.mock_database import mock_database
from magn.magn import MAGNGraph

resources_dir: Final[Path] = Path('../resources/data/')
zip_data_file: Final[Path] = resources_dir.joinpath('pitchfork-data.zip')
database_path: Final[Path] = resources_dir.joinpath('database.sqlite')

# database = Database.from_sqlite3(database_path)
database = mock_database()

magn = MAGNGraph.from_database(database)

Processing tables...
Table reviews processed.
Table artists processed.
Table content processed.
Table genres processed.
Table labels processed.
Table years processed.
Tables processed.


In [2]:
x_train = database.create_mock_target('reviews', seed_id=random.randint(0, 1000))
x_train

title  score     author          author_type    pub_date genre  \
reviewId                                                                     
0         Album 1    1.5  aberfeldy  senior staff writer  2020-01-01  rock   
1         Album 2    3.0   aarktica          contrinutor  2021-01-02   pop   
2         Album 3    7.5   aberdeen  senior staff writer  2022-01-03   rap   
3         Album 4    9.0  aceyalone          contributor  2020-01-04  rock   
4         Album 5    5.5  aceyalone  senior staff writer  2022-01-05   pop   

               target  
reviewId               
0               title  
1            pub_date  
2         author_type  
3               genre  
4         author_type

In [3]:
magn.fit(x_train.head(75), 1000, 0.1)

Teaching MAGN...
epoch 0...
epoch 1...
epoch 2...
epoch 3...
epoch 4...
epoch 5...
epoch 6...
epoch 7...
epoch 8...
epoch 9...
epoch 10...
epoch 11...
epoch 12...
epoch 13...
epoch 14...
epoch 15...
epoch 16...
epoch 17...
epoch 18...
epoch 19...
epoch 20...
epoch 21...
epoch 22...
epoch 23...
epoch 24...
epoch 25...
epoch 26...
epoch 27...
epoch 28...
epoch 29...
epoch 30...
epoch 31...
epoch 32...
epoch 33...
epoch 34...
epoch 35...
epoch 36...
epoch 37...
epoch 38...
epoch 39...
epoch 40...
epoch 41...
epoch 42...
epoch 43...
epoch 44...
epoch 45...
epoch 46...
epoch 47...
epoch 48...
epoch 49...
epoch 50...
epoch 51...
epoch 52...
epoch 53...
epoch 54...
epoch 55...
epoch 56...
epoch 57...
epoch 58...
epoch 59...
epoch 60...
epoch 61...
epoch 62...
epoch 63...
epoch 64...
epoch 65...
epoch 66...
epoch 67...
epoch 68...
epoch 69...
epoch 70...
epoch 71...
epoch 72...
epoch 73...
epoch 74...
epoch 75...
epoch 76...
epoch 77...
epoch 78...
epoch 79...
epoch 80...
epoch 81...
epoch 82.

In [4]:
from magn.prediction_type import PredictionType


def predict_categorical(x_data: pd.DataFrame):
    x_test_no_target = x_data.drop("target")
    prediction = magn.predict(x_test_no_target, 'score', PredictionType.CATEGORICAL)
    print(f"Predicted score: {prediction}, should be {x_data['score']}")
    return prediction == x_data['score']


def predict_regression(x_data: pd.DataFrame):
    x_test_no_target = x_data.drop("target")
    prediction = magn.predict(x_test_no_target, 'score', PredictionType.REGRESSION)
    print(f"Predicted score: {prediction}, should be {x_data['score']}")
    return abs(prediction - x_data['score'])

In [5]:
results = []
for _, row in x_train.iterrows():
    results.append(predict_categorical(row))
print(f"Accuracy: {sum(results) / len(results) * 100.0}%")

Predicted score: 1.5, should be 1.5
Predicted score: 3.0, should be 3.0
Predicted score: 1.5, should be 7.5
Predicted score: 1.5, should be 9.0
Predicted score: 1.5, should be 5.5
Accuracy: 40.0%


In [6]:
results = []
for _, row in x_train.iterrows():
    results.append(predict_regression(row))
print(f"MAE: {sum(results) / len(results)}")

Predicted score: 4.5, should be 1.5
Predicted score: 9.0, should be 3.0
Predicted score: 1.5, should be 7.5
Predicted score: 1.5, should be 9.0
Predicted score: 4.5, should be 5.5
MAE: 4.7
